In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import re

pd.set_option('display.max_columns', None)

In [2]:
dane = pd.read_csv('dane_gotowe.csv', low_memory=False)
dane['is_business'] = dane['is_business'].astype('bool')

city_disrtict_1 = dane['lon_y']
city_disrtict_2 = dane['lat_y']

city_disrtict_1[city_disrtict_1.isnull()] = dane['lon_x'][city_disrtict_1.isnull()]
city_disrtict_2[city_disrtict_2.isnull()] = dane['lat_x'][city_disrtict_2.isnull()]

dane.drop(columns=['Unnamed: 0', 'CityDistrict', 'CityDistrict', 'price[currency]', 'rent[currency]', 'lon_y', 'lat_x', 'lon_x', 'lat_y'], inplace=True)

dane['city_disrtict_1'] = city_disrtict_1
dane['city_disrtict_2'] = city_disrtict_2

mieszkania = dane[dane['category'] == 'Mieszkanie na sprzedaż']
domy = dane[dane['category'] == 'Dom na sprzedaż']

mieszkania.drop(columns=['category'], inplace=True)
domy.drop(columns=['category'], inplace=True)


mieszkania = mieszkania.dropna(axis=1, thresh= np.floor(len(mieszkania)*0.75) )
domy = domy.dropna(axis=1, thresh= np.floor(len(domy)*0.75) )

C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\2739664356.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_disrtict_1[city_disrtict_1.isnull()] = dane['lon_x'][city_disrtict_1.isnull()]
C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\2739664356.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_disrtict_2[city_disrtict_2.isnull()] = dane['lat_x'][city_disrtict_2.isnull()]
C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\2739664356.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [57]:
mieszkania_kopia = mieszkania.copy()
domy_kopia = domy.copy()

In [58]:
mieszkania_kopia.loc[((mieszkania_kopia['m']<10) & (mieszkania_kopia['rooms_num']!='1')), 'm'] = mieszkania_kopia[(mieszkania_kopia['m'] <= 10) & (mieszkania_kopia['rooms_num'] != '1')]['m']*10

do_wyrzucenia = mieszkania_kopia[(mieszkania_kopia['m'] < 10) & (mieszkania_kopia['price'] < 50000)].index

mieszkania_kopia.drop(index=do_wyrzucenia, inplace=True)

In [59]:
mieszkania_kopia[mieszkania_kopia['m']<10]

,created_at_first,m,price,is_business,market,rooms_num,title,building_type,floor_no,building_floors_num,extras_types_balcony,extras_types_garage,media_types_internet,extras_types_basement,extras_types_lift,security_types_entryphone,media_types_cable-television,media_types_phone,is_bungalow,security_types_anti_burglary_door,security_types_monitoring,security_types_closed_area,media_types_electricity,extras_types_separate_kitchen,media_types_water,equipment_types_furniture,media_types_gas,access_types_asphalt,media_types_sewage,extras_types_usable_room,recreational,heating_types_gas,equipment_types_stove,equipment_types_oven,extras_types_terrace,equipment_types_fridge,extras_types_garden,security_types_alarm,access_types_hard_surfaced,equipment_types_washing_machine,vicinity_types_forest,equipment_types_dishwasher,fence_types_wire,media_types_cesspool,security_types_roller_shutters,extras_types_attic,fence_types_metal,heating_types_fireplace,heating_types_coal,extras_types_air_conditioning,media_types_cable_television,extras_types_two_storey,fence_types_brick,equipment_types_tv,fence_types_other,vicinity_types_lake,fence_types_wooden,fence_types_concrete,fence_types_hedge,media_types_water_purification,heating_types_urban,vicinity_types_mountains,heating_types_electric,heating_types_oil,heating_types_heat_pump,access_types_dirt,heating_types_stove,vicinity_types_sea,extras_types_pool,heating_types_solar_collector,heating_types_biomass,heating_types_geothermal,extras_types_heating,rent_to_students,extras_types_elevator,extras_types_asphalt_access,access_types_soft_surfaced,use_types_hotel,use_types_gastronomy,use_types_office,vicinity_types_open_terrain,extras_types_furniture,extras_types_parking,deposit,city_disrtict_1,city_disrtict_2
27029,20181209,6.00,1200000,False,secondary,1,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,16.20152,51.40026
172487,20180623,7.42,99000,True,secondary,1,"Kraków , Swoszowice Nie Przegap - nowa kawalerka",NaN,garret,1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,19.93967,49.97662
188574,20180704,9.50,129000,True,secondary,1,Kawalerka ul Długosza,tenement,floor_2,2.0,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,19.94331,50.03765
192589,20190227,6.80,86000,False,secondary,1,mikroapartament,block,floor_5,5.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,21.02912,52.13785
227568,20181102,8.30,99000,True,primary,1,Studio | Swo

In [54]:
mieszkania_ponizej_10 = mieszkania_kopia[mieszkania.m < 10]
mieszkania_ponizej_10['m'] = mieszkania_ponizej_10['m'] * 10

#mieszkania_ponizej_10.select_dtypes(exclude='bool').rooms_num.value_counts()

mieszkania_ponizej_10

C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\1777527288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mieszkania_ponizej_10['m'] = mieszkania_ponizej_10['m'] * 10


,created_at_first,m,price,is_business,market,rooms_num,title,building_type,floor_no,building_floors_num,extras_types_balcony,extras_types_garage,media_types_internet,extras_types_basement,extras_types_lift,security_types_entryphone,media_types_cable-television,media_types_phone,is_bungalow,security_types_anti_burglary_door,security_types_monitoring,security_types_closed_area,media_types_electricity,extras_types_separate_kitchen,media_types_water,equipment_types_furniture,media_types_gas,access_types_asphalt,media_types_sewage,extras_types_usable_room,recreational,heating_types_gas,equipment_types_stove,equipment_types_oven,extras_types_terrace,equipment_types_fridge,extras_types_garden,security_types_alarm,access_types_hard_surfaced,equipment_types_washing_machine,vicinity_types_forest,equipment_types_dishwasher,fence_types_wire,media_types_cesspool,security_types_roller_shutters,extras_types_attic,fence_types_metal,heating_types_fireplace,heating_types_coal,extras_types_air_conditioning,media_types_cable_television,extras_types_two_storey,fence_types_brick,equipment_types_tv,fence_types_other,vicinity_types_lake,fence_types_wooden,fence_types_concrete,fence_types_hedge,media_types_water_purification,heating_types_urban,vicinity_types_mountains,heating_types_electric,heating_types_oil,heating_types_heat_pump,access_types_dirt,heating_types_stove,vicinity_types_sea,extras_types_pool,heating_types_solar_collector,heating_types_biomass,heating_types_geothermal,extras_types_heating,rent_to_students,extras_types_elevator,extras_types_asphalt_access,access_types_soft_surfaced,use_types_hotel,use_types_gastronomy,use_types_office,vicinity_types_open_terrain,extras_types_furniture,extras_types_parking,deposit,city_disrtict_1,city_disrtict_2
27029,20181209,60.0,1200000,False,secondary,1,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,16.20152,51.40026
29108,20190111,715.0,413885,True,primary,3,"Mieszkanie Na Sprzedaż - Katowice, Kostuchna",apartment,floor_4,4.0,True,False,True,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,18.98193,50.18990
118553,20180823,400.0,144000,False,secondary,2,rfref,tenement,ground_floor,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,15.98862,54.00229
127407,20190227,100.0,10000,False,primary,more,Rakowicka/16mieszkań/Inwestycyjnie,infill,NaN,3.0,True,False,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,19.96044,50.07960
131623,20190313,600.0,665000,True,secondary,3,3-pokojo

In [55]:
do_wyrzucenia = mieszkania_ponizej_10.select_dtypes(exclude='bool')[mieszkania_ponizej_10['price'] < 50000].index #ponieważ to są jakies komorki miejsca parkingowe itd...

mieszkania_kopia.drop(axis=0, index=do_wyrzucenia, inplace=True)

In [56]:
#mieszkania_kopia.select_dtypes(exclude='bool').describe()
#mieszkania_kopia[mieszkania_kopia['price']<50000].describe()
#mieszkania_kopia[(mieszkania_kopia['price']<50000) & (mieszkania_kopia['m'] < 25)]
 
powyzej_1000 = mieszkania_kopia[mieszkania_kopia['m'] > 1000]   ### nie zwiazane w z mieszkaniami

powyzej_1000['m'] = powyzej_1000['m']/100

do_wyrzucenia = powyzej_1000[(powyzej_1000['m']>100) & (powyzej_1000['price']>700000)].index

mieszkania_kopia.drop(axis=0, index=do_wyrzucenia, inplace=True)


######### ZASTANÓW SIĘ NAD TYM!!!! ##############

C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\3407717966.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powyzej_1000['m'] = powyzej_1000['m']/100


In [8]:
mieszkania_kopia.loc[mieszkania_kopia['m'] > 1000, 'm'] = mieszkania_kopia.loc[mieszkania_kopia['m'] > 1000, 'm']/100

do_wyrzucenia = mieszkania_kopia[mieszkania_kopia['m'] > 1000].index
mieszkania_kopia.drop(axis=0, index=do_wyrzucenia, inplace=True)                      ###błedy albo jakies działki bądź inne dziwne rzeczy

#mieszkania.drop

#mieszkania_kopia.loc[mieszkania_kopia['m'] > 1000, 'm'] = mieszkania_kopia.loc[mieszkania_kopia['m'] > 1000, 'm']/100

In [9]:
mieszkania.select_dtypes(exclude= 'bool').describe()

,created_at_first,m,price,building_floors_num,city_disrtict_1,city_disrtict_2
count,6.548410e+05,6.548410e+05,6.548410e+05,597384.000000,654841.000000,654841.000000
mean,2.018268e+07,8.848048e+01,3.853356e+05,5.040319,19.124418,51.780224
std,3.864988e+03,3.743769e+03,3.876230e+05,83.921762,2.007435,1.341454
min,2.018020e+07,1.000000e+00,5.000000e+03,1.000000,14.206510,22.510000
25%,2.018052e+07,4.300000e+01,2.350000e+05,3.000000,17.081270,50.840510
50%,2.018082e+07,5.360000e+01,3.220000e+05,4.000000,19.104980,52.102060
75%,2.018120e+07,6.725000e+01,4.414000e+05,5.000000,20.967270,52.328990
max,2.019033e+07,1.047996e+06,1.000000e+08,52019.000000,51.223610,54.831170


In [10]:
do_wyrzucenia = mieszkania_kopia[(mieszkania_kopia['m'] > 500) & (mieszkania_kopia['price'] < 600000)].index
mieszkania_kopia.drop(axis=0, index=do_wyrzucenia, inplace=True)

In [11]:
#mieszkania_kopia[mieszkania_kopia['m'] > 1000]
pattern = r'\b[kK]amienica\b'
matches = mieszkania_kopia['title'].str.contains(pattern, flags=re.IGNORECASE)
matches.fillna(False, inplace=True)

do_wyrzucenia = mieszkania_kopia[(matches == False) & (mieszkania_kopia['m'] > 1000)].index
mieszkania_kopia.drop(index=do_wyrzucenia, inplace=True)

C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\3648239626.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches.fillna(False, inplace=True)


In [12]:
pattern = r'\b[aA]partament\b'
matches = mieszkania_kopia['title'].str.contains(pattern)

do_wyrzucenia = mieszkania_kopia[(mieszkania_kopia['m'] < 60) & (mieszkania_kopia['price']>1000000) & (matches == False)].index

mieszkania_kopia.drop(index=do_wyrzucenia, inplace=True)

In [13]:
do_wyrzucenia = mieszkania_kopia[mieszkania_kopia['m'] < 5].index

mieszkania_kopia.drop(index=do_wyrzucenia, inplace=True)

In [14]:
mieszkania_kopia.drop_duplicates(inplace=True)

In [15]:
do_wyrzucenia = mieszkania_kopia[mieszkania_kopia['building_floors_num'] > 55].index
mieszkania_kopia.drop(index=do_wyrzucenia, inplace=True)

In [16]:
np.sum(mieszkania_kopia.isnull().sum() != 0)

6

In [18]:
mieszkania_kopia.drop(columns=['title'], inplace=True)

In [19]:
def uzupelnianie_brakow(df):
    df = df[pd.DataFrame(df.isnull().sum().sort_values()).index]



    for i in range(1,len(df.columns)): #zakładamy że chociaż jedna kolumna nie ma braków!
        
        if df.iloc[:,i].isnull().sum() != 0:
            X = df.iloc[:,0:i]
            y = df.iloc[:,i]

            X = pd.get_dummies(X)
            X = X.values

            y_proba_tylko_pelne = y[y.isnull() == False]
            X_proba_tylko_pelne = X[y.isnull() == False]
            X_wybrakowane = X[y.isnull()]



            if y.dtype == 'O' or y.dtype == 'bool':
                las_uzupelniajacy = RandomForestClassifier(n_estimators=100, max_features='sqrt', n_jobs=-1)
                las_uzupelniajacy.fit(X_proba_tylko_pelne, y_proba_tylko_pelne)
                uzupelnione = las_uzupelniajacy.predict(X = X_wybrakowane)

                df.iloc[:,i][df.iloc[:,i].isnull()] = uzupelnione

            else:
                las_uzupelniajacy = RandomForestRegressor(n_estimators=100, max_features='log2', n_jobs=-1)
                las_uzupelniajacy.fit(X_proba_tylko_pelne, y_proba_tylko_pelne)
                uzupelnione = las_uzupelniajacy.predict(X = X_wybrakowane)

                df.iloc[:,i][df.iloc[:,i].isnull()] = uzupelnione

    
        

    return df

wynik = uzupelnianie_brakow(mieszkania_kopia)

C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\1068543035.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.iloc[:,i][df.iloc[:,i].isnull()] = uzupelnione
C:\Users\zbugo\AppData\Local\Temp\ipykernel_20260\1068543035.py:26: SettingWi

In [23]:
wynik_kopia = wynik.copy()
wynik_kopia = pd.get_dummies(wynik_kopia)


y = wynik_kopia.pop('price')
X = wynik_kopia.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

las_kolejny = RandomForestRegressor(n_estimators=100, max_features='log2', n_jobs=-1)
las_kolejny.fit(X=X_train, y=y_train)

RandomForestRegressor(max_features='log2', n_jobs=-1)

In [24]:
predykcja = las_kolejny.predict(X=X_test)

In [25]:
mean_squared_error(y_pred=predykcja, y_true=y_test)

47731411378.32946

In [26]:
mean_absolute_error(y_pred=predykcja, y_true=y_test)

52580.54806284939

In [27]:
mean_absolute_percentage_error(y_pred=predykcja, y_true=y_test)

0.16601452336395262

In [29]:
np.mean(y_test)

385249.75904724794